## Understanding of "fake" sample in test set
dig into deeper what #yag320 mean in the unique occurence in the test set.
https://www.kaggle.com/yag320/list-of-fake-samples-and-public-private-lb-split/comments

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook as tqdm
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.


['train.csv', 'sample_submission.csv', 'test.csv']


In [3]:
df_test = pd.read_csv('../input/test.csv')

In [4]:
df_test.drop(['ID_code'], axis=1, inplace=True)

from the fake row npy file we know that:...

fake rows are:
array([ 0,  1,  2,  4,  5,  6,  8,  9, 10, 12, 13, 14, 19, 23, 25, 26, 27,
       28, 30, 31])


In [5]:
df_test.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,-2.0248,-4.3554,13.9696,0.3458,7.5408,14.5001,7.7028,-19.0919,15.5806,16.1763,3.7088,18.8064,1.5899,3.0654,6.4509,14.1192,-9.4902,-2.1917,5.7107,3.7864,-1.7981,9.2645,2.0657,12.7753,11.3334,8.1462,-0.0610,3.5331,9.7804,8.7625,...,5.9232,5.4113,3.8302,5.7380,-8.6105,22.9530,2.5531,-0.2836,4.3416,5.1855,4.2603,1.6779,29.0849,8.4685,18.1317,12.2818,-0.6912,10.2226,-5.5579,2.2926,-4.5358,10.3903,-15.4937,3.9697,31.3521,-1.1651,9.2874,-23.5705,13.2643,1.6591,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,-1.3809,-0.3310,14.1129,2.5667,5.4988,14.1853,7.0196,4.6564,29.1609,0.0910,12.1469,3.1389,5.2578,2.4228,16.2064,13.5023,-5.2341,-3.6648,5.7080,2.9965,-10.4720,11.4938,-0.9660,15.3445,10.6361,0.8966,6.7428,2.3421,12.8678,-1.5536,...,30.9641,5.6723,3.6873,13.0429,-10.6572,15.5134,3.2185,9.0535,7.0535,5.3924,-0.7720,-8.1783,29.9227,-5.6274,10.5018,9.6083,-0.4935,8.1696,-4.3605,5.2110,0.4087,12.0030,-10.3812,5.8496,25.1958,-8.8468,11.8263,-8.7112,15.9072,0.9812,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,-4.7057,-3.0422,13.6751,3.8183,10.8535,14.2126,9.8837,2.6541,21.2181,20.8163,12.4666,12.3696,4.7473,2.7936,5.2189,13.5670,-15.4246,-0.1655,7.2633,3.4310,-9.1508,9.7320,3.1062,22.3076,11.9593,9.9255,4.0702,4.9934,8.0667,0.8804,...,39.3654,5.5228,3.3159,4.3324,-0.5382,13.3009,3.1243,-4.1731,1.2330,6.1513,-0.0391,1.4950,16.8874,-2.9787,27.4035,15.8819,-10.9660,15.6415,-9.4056,4.4611,-3.0835,8.5549,-2.8517,13.4770,24.4721,-3.4824,4.9178,-2.0720,11.5390,1.1821,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,0.0095,-5.0659,14.0526,13.5010,8.7660,14.7352,10.0383,-15.3508,2.1273,21.4797,14.5372,12.5527,2.9707,4.2398,13.7796,14.1408,1.0061,-1.3479,5.2570,6.5911,6.2161,9.5540,2.3628,10.2124,10.8047,-2.5588,6.0720,3.2613,16.5632,8.8336,...,19.7251,5.3882,3.6775,7.4753,-11.0780,24.8712,2.6415,2.2673,7.2788,5.6406,7.2048,3.4504,2.4130,11.1674,14.5499,10.6151,-5.7922,13.9407,7.1078,1.1019,9.4590,9.8243,5.9917,5.1634,8.1154,3.6638,3.3102,-19.7819,13.4499,1.3104,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,5.1025,-3.2827,14.1013,8.9672,4.7276,14.5811,11.8615,3.1480,18.0126,13.8006,1.6026,16.3059,6.7954,3.6015,13.6569,13.8807,8.6228,-2.2654,5.2255,7.0165,-15.6961,10.6239,-4.7674,17.5447,11.8668,3.0154,4.2546,6.7601,5.9613,0.3695,...,22.8700,5.6688,6.1159,13.2433,-11.9785,26.2040,3.2348,-5.5775,5.7036,6.1717,-1.6039,-2.4866,17.2728,2.3640,14.0037,12.9165,-12.0311,10.1161,-8.7562,6.0889,-1.3620,10.3559,-7.4915,9.4588,3.9829,5.8580,8.3635,-24.8254,11.4928,1.6321,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


## let compare 1 fake row and 1 real row
#### fake: row 0
#### real: row 3

In [6]:
df_test = df_test.values #convert to numpy array

In [7]:
unique_samples = []
unique_count = np.zeros_like(df_test) #create a same shape (200000,200) of zero np

In [8]:
unique_count

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

by observing 1 column.

value 0.1887 occur twice in column var_01 (14030 & 19561)

the value which occur only once, we mark it in unique_count

In [9]:
for feature in tqdm(range(df_test.shape[1])):
    # loop thru each column, find which rows has value that never repeat in the same column and +1 in unique_count
    
    # return the indexes and counts on each unique value found in the column 
    _, index_, count_ = np.unique(df_test[:, feature], return_counts=True, return_index=True)
    
    ## uncomment to see what np.unique return.
    #print (index_[:20])
    #print (count_[:20])
    #break
    
    ## [ 14030 119481 194633 103891  23396 141831 111894  64543  32828 186765  188883 109095  17040 112895   2680 152511  14952  50619   7246  71156]
    ## [2 2 1 1 2 1 1 1 2 1 1 2 4 1 3 1 1 2 2 2]
    
    # meaning at index 14030 , the value 0.1887 repeated twice. (another 1 is index 19561)
    
    
    ### the following line  only +1 those row with 1 occurrence indexes
    unique_count[index_[count_ == 1], feature] += 1

In [10]:
unique_count.shape

(200000, 200)

#### observe row 3, column 1 has a unique value that never occur in column one

In [11]:
unique_count[0:100,0:10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 1., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 

### sum each row, 
if sum greater than 1 => real sample

if sum == 0  => fake sample

In [12]:
# Samples which have unique values are real the others are fake
real_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synthetic_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]

In [13]:
real_samples_indexes

array([     3,      7,     11, ..., 199995, 199996, 199999])

In [14]:
synthetic_samples_indexes

array([     0,      1,      2, ..., 199994, 199997, 199998])

The rest is about detecting which sample is use to generate the public leader board and private leader board which i will **NOT** go thru

In [15]:
# df_test_real = df_test[real_samples_indexes].copy()

# generator_for_each_synthetic_sample = []
# # Using 20,000 samples should be enough. 
# # You can use all of the 100,000 and get the same results (but 5 times slower)
# for cur_sample_index in tqdm(synthetic_samples_indexes[:20000]):
#     cur_synthetic_sample = df_test[cur_sample_index]
#     potential_generators = df_test_real == cur_synthetic_sample
    
#     #print (cur_synthetic_sample)
#     #print (potential_generators)
    
#     # A verified generator for a synthetic sample is achieved
#     # only if the value of a feature appears only once in the
#     # entire real samples set
#     features_mask = np.sum(potential_generators, axis=0) == 1
#     verified_generators_mask = np.any(potential_generators[:, features_mask], axis=1)
#     verified_generators_for_sample = real_samples_indexes[np.argwhere(verified_generators_mask)[:, 0]]
#     generator_for_each_synthetic_sample.append(set(verified_generators_for_sample))

In [16]:
# public_LB = generator_for_each_synthetic_sample[0]
# for x in tqdm(generator_for_each_synthetic_sample):
#     if public_LB.intersection(x):
#         public_LB = public_LB.union(x)

# private_LB = generator_for_each_synthetic_sample[1]
# for x in tqdm(generator_for_each_synthetic_sample):
#     if private_LB.intersection(x):
#         private_LB = private_LB.union(x)
        
# print(len(public_LB))
# print(len(private_LB))